# Production vs Research

Broadly speaking, data science projects can be divided into two categories: research and production. Research projects are intended to investigate some structure or patterns or relationships within a set of data and present the results, or perhaps to develop some new algorithm or technology. Research projects are generally one-shot projects -- we conduct the study, present the results, publish them, and move on to the next project.

Production data science is different. Yes, production builds on research, but research also builds on prior research. But fundamentally, what makes the entire production workflow different from the research workflow is that production models are never really finished. Once a model is put into production, it needs to be maintained, improved, and updated for as long as that model is in use.

In this lecture we will look at principles and techniques you can use to keep your production models performing well, and even better than when you originally train them.

## The Production Lifecycle

During the research phase, your workflow will follow (more or less) this pattern:
- Ask a question
- Collect data
- Clean data
- Conduct exploratory data analysis
- Construct a (specific) hypothesis
- Choose a model and algorithm
- Engineer features
- Train the model
- Evaluate the model
- Write and publish results

For a production model, however, once the model has been deployed in an application, you will notice something terrible.

Your model will no longer work the way you expected it to.

This is because in a live production environment, you will begin to see data that was not in your training or test sets. Users will ask your model questions that you did not anticipate during the training phase. Your code will have bugs. All of this is completely fine. (It's good, in fact, because it means you will still have work to do. Job security is a wonderful thing.)

So compared to the research workflow, the production lifecycle looks like this:
- ... earlier steps as before ...
- Engineer features
- Train the model
- Evaluate the model
- Deploy (publish) the model
- Collect usage data
- Obtain labels for usage data *(this step is left as an exercise for the reader)*
- Retrain the model
- Evaluate the new model
- Deploy the new model
- Collect usage data ...

and so on. And it gets better -- you may decide that you need to redesign and train a new model, which will hopefully take the same inputs and yield the same outputs so that it doesn't break your production pipeline.

## Iterative Deployments

All of this means that for data scientists working on deployed models, their daily responsibilities involve
- monitoring the model's performance
- retraining the model with new user data
- searching for common failures with a consistent pattern
- redesigning the model and retraining to improve performance in those cases
- comparing new model performance to old performance to ensure there are no regressions*
- running unit tests on the new model (and passing it through a code review process)
- deploying the model update

We've worked with unit tests and designing and training models before. What's new here is that now we need to start collecting data from the model's usage in the wild.

\* *You're going to have to get used to seeing the word "regression" used differently by data scientists and engineers. When we're talking about software updates, a regression is a decrease in performance compared to the old iteration.*

## User Data Collection: logging in Python

So far, you've been using print statements to check on what your code has been doing at each stage of development. While print() is great for interactive programming, logging is essential for getting feedback from a model in production. Fortunately for us, Python's built-in logging library is easy to use and contains all of the functionality we will need.

The key differences between logging and print are that
- logging contains several general purpose formatting options that make it easy for us to create organized logs
- logging also contains a basic configuration command that makes it easy to specify how the logs will be kept
- logging enables us to categorize our output messages according to increasing levels of severity

Logs are reported according to the severity of the issue being logged. That is up to you to decide, but the Python documentation recommends you use them in these scenarios:

Level	When it’s used
DEBUG	Detailed information, typically of interest only when diagnosing problems.
INFO	Confirmation that things are working as expected.
WARNING	An indication that something unexpected happened, or indicative of some problem in the near future (e.g. ‘disk space low’). The software is still working as expected.
ERROR	Due to a more serious problem, the software has not been able to perform some function.
CRITICAL	A serious error, indicating that the program itself may be unable to continue running.

By default, the logging library will only actually record logs where the severity level is WARNING or greater. But we can configure it to record lower-severity logs as well. In fact, for data collection purposes, we will probably want to log our usage data as INFO logs.

## Using the logging library: an example

Adding logging functionality to your program is simply a matter of importing the logging library, making a function call to set some basic logging parameters, and then replacing your print() statements (or adding new ones) with logging statements that record whatever you will want to track once your model has been deployed.

If your model has been deployed in an app, you will most likely want to record all of the queries that your users are asking your model to predict, as well as the outputs of those predictions. Depending on how your app is implemented, you may also want to keep logs of anything that may indicate how satisfied your users are with those predictions. Otherwise you will have to find other ways to determine whether the predictions were good or not.

Let's work through a simple code sample where a trained model is loaded into our application and a few queries are interactively predicted and the results are printed to the console.


In [ ]:
# load pre-trained model

# get user input
# print input to command line

# use model to predict outcome of user's query
# print output to command line

Now let's revise our code so that instead of printing our inputs and outputs to the command line, we're storing them in a file for analysis at a later date. Note that if we want to use this data to retrain our model, we will need to find a way to get targets for the model to predict from these queries.

In [ ]:
# import logging
# set logging config

# load pre-trained model

# get user input
# log user input

# use model to predict outcome of user's query
# log prediction (maybe linked to logged input?)

## Preparing Data for Retraining: Tagging Queries with Mechanical Turk

Now that you have a system in place for collecting the queries that are being entered into your production model, you're one step away from being able to retrain your model to perform better on live data.

This is not a trivial step, however.

In order to re-train or evaluate your model on live data, you need to know what the "right" outputs are!

There are a few ways you can approach this problem. 

- You can provide a user feedback interface on your app, where users can submit queries with suggestions for improvement. You can either trust that these submissions have correct answers and retrain on them, or you can design a review protocol to make sure that the suggestions are good.
- You can write programs to analyze user actions and look for clues that they are not satisfied with the answers they are receiving, such as repeated variations of the same query until the model gets it "right". Again, this may or may not be obvious depending on context.
- You can collect queries and their predictions (or not even the predictions) and submit them to be evaluated by human judges. These can be members of your team, or they can be outsourced to a service like Amazon's Mechanical Turk or UHRS clickworker.

Mechanical Turk and UHRS are online services that allow you to submit repetitive tasks to be completed by human workers on the other end. These are not volunteers; you will need to decide how much to pay them for each query they tag for you. You may find that you need to increase the price you are offering in order to get enough workers to do the job.

The most important things to consider with tagging* data are accuracy, time, and cost.

\* *Note that you may also hear this process described as labelling, evaluating, annotating, judging, or other similar terms.*

### Accuracy

Before you start tagging your data, or before you can submit it to be tagged by others, you have to be able to provide a very clear description of rules for how the tags should be applied. This is especially important for qualitative tags. (Quantitative tags can be objectively verified but you will need to find a reliable source to check your numbers.) 

For qualitative tags, you will need to come up with a list of rules for how each one should be applied to a given query. Ideally these rules will be exhaustive and mutually exclusive. But depending on the task your model is performing, this probably won't be obvious. If it were, you probably wouldn't need to rely on machine learning to solve this problem. 

With that out of the way, here's a common approach to establishing your guidelines:
- Collect 100 or so queries that need to be tagged.
- Review them yourself, identify the tags that will be needed (categories, identifiers, keywords, etc).
- Write rules that (you think) are sufficient to correctly tag each query.
- Find 4+ friends or co-workers and have them read your rules and tag 20 queries.
- Collect the results and look for disagreements in the tags.
- Revise your rules, give them to a new group of co-workers or friends, along with 20 new queries.
- Rinse and repeat until you either have a set of rules that yields matching tags, or you don't have any more friends who are willing to help with this project.

Note that the above process is not completely necessary, but doing some work to make sure you have clear guidelines will save you lots of time and money when you submit datasets to be tagged by Mechanical Turk. Keep in mind that the queries that are not being predicted accurately by your model are usually the harder queries to correctly tag, so tagging them with consensus may not be an easy feat to accomplish! Providing clear guidelines is the best thing you can do to ensure success.

### Time and Cost (note that these are really the same thing)

Once your data has been collected and you are confident with your tagging guidelines, you are ready to submit your queries to be tagged. Now you're going to need to figure out how much you are willing to pay to have your data tagged. A good rule of thumb is to decide on an hourly wage you are willing to pay to the Turkers, and divide that by the estimated number of queries you think can be tagged in an hour.

If this is a challenging dataset to tag correctly, and you want to make sure that your queries are being tagged by Turkers who can prove that they have a solid grasp of the rules, you may want to offer a higher price per tag, and provide a test set that Turkers must tag correctly in order to qualify to work on your data set.

# TO DO

Include some diagrams to show how everything fits together
- Ensembling
- HTML
- JavaScript
- Tableau
- Bokeh / Plotly
- Flask
- D3

Provide examples of queries and good or bad guidelines for tagging

Screenshots of setting up a job on Mechanical Turk

Maybe actually deploy a model on Heroku and demonstrate how all the parts work together?